In [1]:
import sys, random, os
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.augment import Augment
import pandas as pd

In [2]:
es_index = "datamart"

augment = Augment(es_index=es_index)

### Initialize a dataframe

In [3]:
old_df = pd.DataFrame(data={
    'city': ["los angeles", "New york", "Shanghai", "SAFDA", "manchester"],
    'country': ["US", "US", "China", "fwfb", "UK"],
})

print(old_df)

          city country
0  los angeles      US
1     New york      US
2     Shanghai   China
3        SAFDA    fwfb
4   manchester      UK


### Search metadata
Query by a column, which is query on variable.named_entities, by default, if a metadata matches more than half of cells in original dataframe, it is a hit

In [4]:
hitted_metadatas = augment.query_by_column(col=old_df.loc[:, "city"])

print(len(hitted_metadatas))

136


In [5]:
# Can also query by other api
hitted_metadatas = augment.query_by_key_value_pairs(key_value_pairs=[
    ("description", "average")
])

print(len(hitted_metadatas))

8


With some ranking methods, say we want to augment with a specific metadata, datamart id 1230000

In [6]:
metadata = augment.query_by_datamart_id(datamart_id=1230000)[0]

### Materialize dataset

In [7]:
new_df = augment.get_dataset(metadata=metadata, variables=None, constrains={
            "locations": old_df.loc[:, 'city'].unique().tolist(),
            "date_range": {
                "start": "2018-09-23T00:00:00",
                "end": "2018-09-30T00:00:00"
            }
        })

In [8]:
print(new_df.iloc[random.sample(range(1, new_df.shape[0]), 10), :])

                    date          stationid         city  TAVG
63   2018-09-27T00:00:00  GHCND:USW00023129  los angeles   213
138  2018-09-28T00:00:00  GHCND:CHM00058367     Shanghai   239
135  2018-09-27T00:00:00  GHCND:CHM00058362     Shanghai   234
9    2018-09-23T00:00:00  GHCND:USR0000CSFD  los angeles   222
96   2018-09-30T00:00:00  GHCND:USR0000CCP9  los angeles   188
80   2018-09-29T00:00:00  GHCND:USR0000CCHB  los angeles   191
119  2018-09-28T00:00:00  GHCND:USW00014732     New york   174
77   2018-09-28T00:00:00  GHCND:USW00023174  los angeles   195
150  2018-09-30T00:00:00  GHCND:USW00014745   manchester   107
91   2018-09-30T00:00:00  GHCND:USR0000CACT  los angeles   201


### Join
There are many ways of join between original dataframe and new dataframe.
Simplest solution if right join which will produce lots of rows for same city.

In [9]:
df = pd.merge(left=old_df, right=new_df, left_on='city', right_on='city', how='right')
print(df)

            city country                 date          stationid  TAVG
0    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CACT   233
1    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CBEV   206
2    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCHB   228
3    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCHI   218
4    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCLE   237
5    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCP9   224
6    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CLTU   215
7    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CMAL   197
8    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CMIL   212
9    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CSFD   222
10   los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CWHH   206
11   los angeles      US  2018-09-23T00:00:00  GHCND:USW00023129   214
12   los angeles      US  2018-09-23T00:00:00  GHCND:USW00023174   201
13   l

#### Aggregation
Join also can be performed based on aggregation.

In [10]:
new_df_aggregated = new_df.groupby(["city", "date"], as_index=False)["TAVG"].mean()
print(new_df_aggregated)

           city                 date        TAVG
0      New york  2018-09-23T00:00:00  174.000000
1      New york  2018-09-24T00:00:00  176.333333
2      New york  2018-09-25T00:00:00  184.666667
3      New york  2018-09-26T00:00:00  239.333333
4      New york  2018-09-27T00:00:00  206.666667
5      New york  2018-09-28T00:00:00  170.000000
6      New york  2018-09-29T00:00:00  182.000000
7      New york  2018-09-30T00:00:00  174.333333
8      Shanghai  2018-09-23T00:00:00  254.500000
9      Shanghai  2018-09-24T00:00:00  210.000000
10     Shanghai  2018-09-25T00:00:00  233.000000
11     Shanghai  2018-09-26T00:00:00  234.500000
12     Shanghai  2018-09-27T00:00:00  236.000000
13     Shanghai  2018-09-28T00:00:00  239.000000
14     Shanghai  2018-09-29T00:00:00  232.500000
15     Shanghai  2018-09-30T00:00:00  232.000000
16  los angeles  2018-09-23T00:00:00  216.384615
17  los angeles  2018-09-24T00:00:00  195.923077
18  los angeles  2018-09-25T00:00:00  202.923077
19  los angeles  201

In [11]:
df = pd.merge(left=old_df, right=new_df_aggregated, left_on='city', right_on='city', how='right')
print(df)

           city country                 date        TAVG
0   los angeles      US  2018-09-23T00:00:00  216.384615
1   los angeles      US  2018-09-24T00:00:00  195.923077
2   los angeles      US  2018-09-25T00:00:00  202.923077
3   los angeles      US  2018-09-26T00:00:00  227.538462
4   los angeles      US  2018-09-27T00:00:00  238.692308
5   los angeles      US  2018-09-28T00:00:00  212.615385
6   los angeles      US  2018-09-29T00:00:00  192.692308
7   los angeles      US  2018-09-30T00:00:00  209.692308
8      New york      US  2018-09-23T00:00:00  174.000000
9      New york      US  2018-09-24T00:00:00  176.333333
10     New york      US  2018-09-25T00:00:00  184.666667
11     New york      US  2018-09-26T00:00:00  239.333333
12     New york      US  2018-09-27T00:00:00  206.666667
13     New york      US  2018-09-28T00:00:00  170.000000
14     New york      US  2018-09-29T00:00:00  182.000000
15     New york      US  2018-09-30T00:00:00  174.333333
16     Shanghai   China  2018-0

We can also unstack new datagrame to form more columns and so that we will not produce extra rows

In [12]:
new_df_unstacked = new_df.groupby(["city", "date"])["TAVG"].mean().unstack().reset_index(level=['city'])
print(new_df_unstacked)

date         city  2018-09-23T00:00:00  2018-09-24T00:00:00  \
0        New york           174.000000           176.333333   
1        Shanghai           254.500000           210.000000   
2     los angeles           216.384615           195.923077   
3      manchester           113.000000           103.000000   

date  2018-09-25T00:00:00  2018-09-26T00:00:00  2018-09-27T00:00:00  \
0              184.666667           239.333333           206.666667   
1              233.000000           234.500000           236.000000   
2              202.923077           227.538462           238.692308   
3               92.000000           198.000000           187.000000   

date  2018-09-28T00:00:00  2018-09-29T00:00:00  2018-09-30T00:00:00  
0              170.000000           182.000000           174.333333  
1              239.000000           232.500000           232.000000  
2              212.615385           192.692308           209.692308  
3              144.000000           128.000000  

In [13]:
df = pd.merge(left=old_df, right=new_df_unstacked, left_on='city', right_on='city')
print(df)

          city country  2018-09-23T00:00:00  2018-09-24T00:00:00  \
0  los angeles      US           216.384615           195.923077   
1     New york      US           174.000000           176.333333   
2     Shanghai   China           254.500000           210.000000   
3   manchester      UK           113.000000           103.000000   

   2018-09-25T00:00:00  2018-09-26T00:00:00  2018-09-27T00:00:00  \
0           202.923077           227.538462           238.692308   
1           184.666667           239.333333           206.666667   
2           233.000000           234.500000           236.000000   
3            92.000000           198.000000           187.000000   

   2018-09-28T00:00:00  2018-09-29T00:00:00  2018-09-30T00:00:00  
0           212.615385           192.692308           209.692308  
1           170.000000           182.000000           174.333333  
2           239.000000           232.500000           232.000000  
3           144.000000           128.000000       